In [ ]:
# By default, the TPU image has jax==0.3.25, jaxlib==0.3.25, flax==0.6.11
# But, flax 0.6.11 actually requires jax>=0.4.2 as a dependency:
# https://github.com/google/flax/blob/fd567c8578f2ba346321890aa1657fce93b4eecf/pyproject.toml#L14
# Not sure how this happened!  The solution is
# !pip show jax jaxlib flax

# This for some reason also updates jax as a side-effect, even though flax==0.6.4
# only requires jax>=0.3.16
# https://github.com/google/flax/blob/dc28d8cbdea8dfbd24868d2b528cd8dc47c4df7e/setup.py#L29
# !pip install flax==0.6.4


# Then this - see https://github.com/google/flax/issues/2950#issuecomment-1490223629

# The solution is this incantation:
!pip install flax==0.6.4 jax==0.3.25 jaxlib==0.3.25 orbax==0.1.0
!pip install --upgrade git+https://github.com/hrbigelow/streamvis.git

In [ ]:
# !pip uninstall -y streamvis
!pip uninstall -y transformer-aiayn
!pip install --upgrade git+https://github.com/hrbigelow/transformer-aiayn.git
# you then need to restart the runtime (Ctrl-M <period>)
# !pip install --upgrade streamvis
# !git clone https://github.com/hrbigelow/transformer-aiayn.git
# !pip install -e transformer-aiayn
# It seems you need to restart runtime after this step


In [ ]:
# Sometimes, this fails with:
# Backend 'tpu_driver' failed to initialize: DEADLINE_EXCEEDED: Failed to connect to remote server at address: grpc://10.6.103.106:8470. Error from gRPC: Deadline Exceeded. Details:
# If this happens, restart with Ctrl-M <period> and rerun cell

import jax.tools.colab_tpu
jax.tools.colab_tpu.setup_tpu()

import jax
jax.devices()
# Should return an array of 8 [TpuDevice(id=...) ...]
# If this returns [CpuDevice()], restart the runtime with Ctrl-M <period> and re-run this cell
# Sometimes it will return CpuDevice after running the previous cell.

In [ ]:
# In order to use Google Cloud Storage (for dataset and checkpoints identified by gs://... globs)
# first create a Google Cloud project and then enable the GCS API
!gcloud config set project <YOUR_PROJECT_ID>

from google.colab import auth
auth.authenticate_user()

Updated property [core/project].


In [ ]:
# %env TF_CPP_MIN_LOG_LEVEL=0
import nest_asyncio
nest_asyncio.apply()

from importlib import reload

# reload(aiayn.train)
import aiayn.train

# Another sporadic error:
# XlaRuntimeError: FAILED_PRECONDITION: Dependency failed: Could not allocate 102940672 bytes in memory 0x0x0_HBM0; 79691776 bytes allocatable, 123109376 bytes available

run = 'sept9'

# Create a bucket (here called ml-checkpoints)
gsroot = 'gs://ml-checkpoints/'

# 
dataset_glob = gsroot + 'de-en-32k/*.tfrecord'
val_dataset_glob = gsroot + 'de-en-val/*.tfrecord'
ckpt_dir = gsroot + 'sept9'

streamvis_run_name = run + '-mask-bug-fix'
streamvis_path = gsroot + 'svlog6'
tokenizer_file = gsroot + 'hf/de-en.bpe.36500.json'
resume_ckpt = 20000


aiayn.train.main(
     'arch,reg,train,data,logging',
     dataset_glob=dataset_glob,
     val_dataset_glob=val_dataset_glob,
     ckpt_every=3000,
     ckpt_dir=ckpt_dir,
     resume_ckpt=resume_ckpt,
     report_every=10,
     batch_dim0=96,
     accum_steps=2,
     val_loop_elem=32,
     eval_every=100,
     max_source_len=320,
     max_target_len=320,
     streamvis_run_name=streamvis_run_name,
     streamvis_buffer_items=500,
     streamvis_path=streamvis_path,
     tokenizer_file=tokenizer_file,
     warmup_steps=4000,
     swap_source_target=True,
     with_metrics=False,
     shuffle_size=100000,
     label_smooth_eps=0.1,
     pos_encoding_factor=1.0,
     attn_loss_weight=0.0
     )

"""
For some reason, launching the program this way results in errors like 'libtpu.so not found'
!/usr/bin/python3 -m aiayn.train ...
"""